
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/01_Storage_vs_View.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 1: Storage vs Tensor (Prawda o Pamięci)

W PyTorch **Tensor to tylko iluzja**. To nakładka (interfejs) na prawdziwe dane.
Prawdziwe dane żyją w obiekcie zwanym **Storage**.

*   **Storage:** Ciągły, jednowymiarowy blok bajtów w pamięci (np. `[1, 2, 3, 4, 5, 6]`).
*   **Tensor:** Zestaw metadanych (Kształt, Stride, Offset), który mówi, jak "czytać" ten blok.

**Dlaczego to kluczowe?**
Operacje takie jak `transpose`, `permute`, `narrow` czy `expand` **nie ruszają danych w pamięci**. Zmieniają tylko metadane. Są błyskawiczne ($O(1)$).
Ale mają swoją cenę: tracą **ciągłość (contiguity)**, co powoduje błędy przy próbie użycia `.view()`.

W tej lekcji zhakujemy pamięć PyTorcha.

In [1]:
import torch
import ctypes

# Funkcja pomocnicza do podglądania adresu pamięci
def print_memory_address(tensor):
    print(f"Adres danych (data_ptr): {tensor.data_ptr()}")

# 1. TWORZYMY TENSOR
t = torch.tensor([[1, 2, 3], [4, 5, 6]])

print("--- TENSOR ---")
print(t)
print(f"Kształt: {t.shape}")
print_memory_address(t)

--- TENSOR ---
tensor([[1, 2, 3],
        [4, 5, 6]])
Kształt: torch.Size([2, 3])
Adres danych (data_ptr): 5829008883840


## Storage: To, co jest pod maską

Zobaczmy, jak te dane wyglądają naprawdę.
Mimo że tensor jest 2D (wiersze i kolumny), w pamięci komputera jest to **płaska lista**.

In [2]:
# Dostęp do surowego magazynu danych
storage = t.untyped_storage()

print("--- STORAGE (Surowe dane) ---")
print(f"Rozmiar storage: {len(storage)} bajtów (6 liczb int64 * 8 bajtów = 48)")
# Podgląd zawartości (jako lista)
print(storage.tolist())

# DOWÓD: Zmiana w Storage zmienia Tensor!
print("\n--- HACKOWANIE PAMIĘCI ---")
# Zmieniamy pierwszy element w storage (fizycznej pamięci)
# Uwaga: Storage jest płaski, więc indeksujemy liniowo
t.untyped_storage()[0] = 99 

print("Tensor po zmianie storage'a:")
print(t)
print("Widzisz? Zmieniła się liczba w tensorze, choć go nie dotykaliśmy.")

--- STORAGE (Surowe dane) ---
Rozmiar storage: 48 bajtów (6 liczb int64 * 8 bajtów = 48)
[1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0]

--- HACKOWANIE PAMIĘCI ---
Tensor po zmianie storage'a:
tensor([[99,  2,  3],
        [ 4,  5,  6]])
Widzisz? Zmieniła się liczba w tensorze, choć go nie dotykaliśmy.


## Metadane: Shape, Offset i Stride

Skąd PyTorch wie, że `t[1, 0]` to liczba `4`?
Używa do tego **Stride (Kroku)**.

*   **Stride:** Ile elementów muszę przeskoczyć w pamięci, żeby zmienić indeks o 1 w danym wymiarze?

Dla tensora `[[1, 2, 3], [4, 5, 6]]`:
*   Żeby przejść do następnego wiersza (dół), muszę przeskoczyć 3 liczby.
*   Żeby przejść do następnej kolumny (prawo), muszę przeskoczyć 1 liczbę.
*   Stride = `(3, 1)`.

In [3]:
# Resetujemy tensor
t = torch.tensor([[1, 2, 3], [4, 5, 6]])

print(f"Tensor:\n{t}")
print(f"Stride: {t.stride()}") 
# (3, 1) -> Skocz o 3, żeby zmienić wiersz. Skocz o 1, żeby zmienić kolumnę.

# MAGIA: Transpozycja
t_transposed = t.t()

print("\n--- PO TRANSPOZYCJI ---")
print(f"Tensor:\n{t_transposed}")
print(f"Stride: {t_transposed.stride()}")
# (1, 3) -> Teraz skocz o 1, żeby zmienić wiersz!

print("\n--- CZY DANE SIĘ PRZESUNĘŁY? ---")
print_memory_address(t)
print_memory_address(t_transposed)

if t.data_ptr() == t_transposed.data_ptr():
    print("✅ Adresy są IDENTYCZNE! Transpozycja nie skopiowała ani jednego bajta.")

Tensor:
tensor([[1, 2, 3],
        [4, 5, 6]])
Stride: (3, 1)

--- PO TRANSPOZYCJI ---
Tensor:
tensor([[1, 4],
        [2, 5],
        [3, 6]])
Stride: (1, 3)

--- CZY DANE SIĘ PRZESUNĘŁY? ---
Adres danych (data_ptr): 5829008883968
Adres danych (data_ptr): 5829008883968
✅ Adresy są IDENTYCZNE! Transpozycja nie skopiowała ani jednego bajta.


## Pułapka: Contiguous vs Non-Contiguous

Transpozycja była "darmowa", ale zapłaciliśmy za nią cenę.
Oryginalny tensor w pamięci wygląda tak: `1, 2, 3, 4, 5, 6`.
Czytając go wierszami (po transpozycji): `1, 4, 2, 5, 3, 6`.

To oznacza, że logiczne następstwo elementów **nie pokrywa się** z ich fizycznym ułożeniem w pamięci.
Tensor jest **nieciągły (Non-Contiguous)**.

Metoda `.view()` działa TYLKO na ciągłych tensorach. Sprawdźmy to.

In [4]:
print(f"Czy oryginał jest ciągły? {t.is_contiguous()}")
print(f"Czy transponowany jest ciągły? {t_transposed.is_contiguous()}")

print("\n--- PRÓBA UŻYCIA .view() ---")
try:
    # Próbujemy spłaszczyć transponowany tensor
    flat = t_transposed.view(-1)
except RuntimeError as e:
    print(f"🚫 BŁĄD: {e}")

print("\n--- ROZWIĄZANIE 1: .contiguous() ---")
# To fizycznie kopiuje dane i układa je poprawnie w nowym miejscu pamięci
t_cont = t_transposed.contiguous()
print(f"Czy teraz ciągły? {t_cont.is_contiguous()}")
print(f"Nowy adres pamięci: {t_cont.data_ptr()} (Inny niż oryginał!)")
print("View działa:", t_cont.view(-1))

print("\n--- ROZWIĄZANIE 2: .reshape() ---")
# reshape() jest mądre: jeśli może zrobić view, robi view. Jeśli nie, robi contiguous() + view.
print("Reshape działa:", t_transposed.reshape(-1))

Czy oryginał jest ciągły? True
Czy transponowany jest ciągły? False

--- PRÓBA UŻYCIA .view() ---
🚫 BŁĄD: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

--- ROZWIĄZANIE 1: .contiguous() ---
Czy teraz ciągły? True
Nowy adres pamięci: 5829008884160 (Inny niż oryginał!)
View działa: tensor([1, 4, 2, 5, 3, 6])

--- ROZWIĄZANIE 2: .reshape() ---
Reshape działa: tensor([1, 4, 2, 5, 3, 6])


## Hardcore: as_strided (Magia Splotów)

Możemy stworzyć tensor "z powietrza", manipulując stride'ami ręcznie.
To jest technika używana do implementacji **Conv2d** (tzw. `im2col`).

Stworzymy "okna" (sliding windows) bez pętli i bez kopiowania pamięci.

In [5]:
# Wektor 1D: [0, 1, 2, 3, 4]
x = torch.arange(5)

# Chcemy uzyskać okna o rozmiarze 2:
# [0, 1]
# [1, 2]
# [2, 3]
# [3, 4]

# Fizycznie w pamięci mamy 5 liczb.
# Stride (1): Żeby przejść w dół (do nast. okna), przesuń się o 1 (0->1).
# Stride (2): Żeby przejść w prawo (wewnątrz okna), przesuń się o 1 (0->1).

windows = x.as_strided(size=(4, 2), stride=(1, 1))

print("--- SLIDING WINDOWS (Zero Copy) ---")
print(windows)
print_memory_address(x)
print_memory_address(windows)
print("To ten sam obszar pamięci! Stworzyliśmy wirtualną macierz.")

--- SLIDING WINDOWS (Zero Copy) ---
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4]])
Adres danych (data_ptr): 5829008884416
Adres danych (data_ptr): 5829008884416
To ten sam obszar pamięci! Stworzyliśmy wirtualną macierz.


## 🥋 Black Belt Summary

1.  **Tensor $\neq$ Pamięć.** Tensor to tylko "okulary", przez które patrzymy na pamięć (Storage).
2.  **Operacje Metadata-only:** `t()`, `permute()`, `transpose()` są super szybkie, bo nie ruszają danych. Zmieniają tylko `stride`.
3.  **Pułapka View:** `.view()` wymaga, żeby dane w pamięci leżały w takiej kolejności, jak sugeruje kształt tensora. Jeśli zrobisz `transpose`, psujesz tę kolejność.
4.  **Naprawa:**
    *   `.contiguous()`: "Uporządkuj mi to fizycznie w pamięci" (Kopiowanie = Wolne).
    *   `.reshape()`: "Zrób co trzeba, żeby zadziałało" (Bezpieczne).

Wydajny kod PyTorch unika `.contiguous()`, jeśli to możliwe, i operuje na widokach.